In [1]:
import os
import re
import MeCab
import pickle
import gensim
import ast
import lda
import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim
import gensim.corpora as corpora
from gensim import similarities 
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
import spacy
from pprint import pprint
import warnings
import logging
from google.cloud import storage as gcs
from sklearn.feature_extraction.text import TfidfTransformer, TfidfVectorizer, CountVectorizer
from sklearn.decomposition import PCA, TruncatedSVD, LatentDirichletAllocation
from sklearn.cluster import KMeans
from sklearn.manifold import TSNE
from sklearn.metrics import accuracy_score, f1_score, fbeta_score
from sklearn.metrics.pairwise import cosine_similarity 
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import seaborn as sns
from sklearn import linear_model
from sklearn import metrics
from sklearn.model_selection import KFold
from scipy.spatial.distance import pdist 
from scipy.spatial.distance import squareform
from bokeh.plotting import figure, output_file, show
from bokeh.models import Label
from bokeh.io import output_notebook
from collections import Counter
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objs as go
%matplotlib inline

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/protobuf/descriptor.py:47: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from google.protobuf.pyext import _message


In [2]:
INPUT_BUCKET = 'gs://mj-labeling-questionnaires-dev.datasets.linkbal.com/inputs/ver_01/'
PROJECT_NAME = 'linkbal-dp'
BUCKET_NAME = 'mj-labeling-questionnaires-dev.datasets.linkbal.com' 
client = gcs.Client(PROJECT_NAME)
bucket = gcs.Bucket(client, name=BUCKET_NAME)
jp_stop_words = open("/Users/jesse.ulundo/demo_tests/Japanese_stopword_list.txt", "r")
qs = pd.read_csv(INPUT_BUCKET + 'mj_questionnaire_text.csv', sep=',')

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/

//anaconda3/envs/jctensorflow/lib/python3.7/site-packages/google/auth/_default.py:69: UserWarning:

Your application has authenticated using end user credentials from Google Cloud SDK. We recommend that most server applications use service accounts instead. If your application continues to use end user credentials from Cloud SDK, you might receive a "quota exceeded" or "API not enabled" error. For more information about service accounts, see https://cloud.google.com/docs/authentication/



## Removing Noise From Data

In [3]:
qs = qs.drop(['want_recommend', 'want_recommend_reason'], axis=1)
qs = qs.dropna()
qs = qs.drop_duplicates() 
qs

,id,overall_satisfaction,overall_satisfaction_reason
4,111376,6.0,とても楽しかったです\nいろんな人と友達になれたのが◎
5,111377,2.0,店舗スタッフの対応が悪い
7,111379,2.0,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...
8,111380,4.0,結果が伴ってなかったから
9,111381,2.0,想像していた人数よりも少ない。
...,...,...,...
20305,131673,3.0,相手の人が少なくて暇な時間がさみしかったです。
20306,131674,3.0,街コンには問題はなかったけど、いい出会いはありませんでした
20307,131675,1.0,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...
20308,131676,2.0,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。


In [4]:
qs = qs[qs.overall_satisfaction < 3]
qs = qs[qs.overall_satisfaction_reason != '上記同様']
qs = qs[qs.overall_satisfaction_reason != '同上']
qs

,id,overall_satisfaction,overall_satisfaction_reason
5,111377,2.0,店舗スタッフの対応が悪い
7,111379,2.0,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...
9,111381,2.0,想像していた人数よりも少ない。
11,111383,2.0,大人数なのにマンツーマン、フリータイムが少ない。
14,111386,2.0,食べ物が酒の肴にならない
...,...,...,...
20295,131663,1.0,参加者が少なすぎる
20296,131664,1.0,料理の品数が少なすぎた。\nスタッフの対応も最悪。
20307,131675,1.0,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...
20308,131676,2.0,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。


## Tokenizing and Stemming

In [5]:
tagger = MeCab.Tagger()
def tokenize_stemmer(text):
    lemmas = []
    for line in tagger.parse(text).splitlines()[:-1]:
        surface, feature = line.split('\t')
        if feature.split(',')[6] != '*':
            lemmas.append(feature.split(',')[6])
    return lemmas

In [6]:
categories = ['参加人数に関して', '男女比率に関して', 'スタッフ対応に関して', '飲食内容に関して', '虚偽記載']
categories_tkn = [tokenize_stemmer(text) for text in categories]
categories_tkn

[['参加', '人数', 'に関して'],
 ['男女', '比率', 'に関して'],
 ['スタッフ', '対応', 'に関して'],
 ['飲食', '内容', 'に関して'],
 ['虚偽', '記載']]

In [7]:
qs['tokenized_overall_satisfaction_reason'] = qs.apply(lambda row: tokenize_stemmer(row['overall_satisfaction_reason']), axis=1)
qs

,id,overall_satisfaction,overall_satisfaction_reason,tokenized_overall_satisfaction_reason
5,111377,2.0,店舗スタッフの対応が悪い,"[店舗, スタッフ, の, 対応, が, 悪い]"
7,111379,2.0,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...,"[他, の, の, お, 店, だ, と, 店員, や, 常連, さん, が, 気, を, ..."
9,111381,2.0,想像していた人数よりも少ない。,"[想像, する, て, いる, た, 人数, より, も, 少ない, 。]"
11,111383,2.0,大人数なのにマンツーマン、フリータイムが少ない。,"[大, 人数, だ, のに, マンツーマン, 、, フリー, タイム, が, 少ない, 。]"
14,111386,2.0,食べ物が酒の肴にならない,"[食べ物, が, 酒, の, 肴, に, なる, ない]"
...,...,...,...,...
20295,131663,1.0,参加者が少なすぎる,"[参加, 者, が, 少ない, すぎる]"
20296,131664,1.0,料理の品数が少なすぎた。\nスタッフの対応も最悪。,"[料理, の, 品数, が, 少ない, すぎる, た, 。, スタッフ, の, 対応, も,..."
20307,131675,1.0,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...,"[最終, 的, に, 社長, から, 返金, 対応, する, 旨, の, 提案, を, 受け..."
20308,131676,2.0,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。,"[水上, バス, は, 夜景, が, きれい, だ, た, けど, 、, ほんとに, それ,..."


In [8]:
stop = [words.strip("\n") for words in jp_stop_words]
qs['tokenized_overall_satisfaction_reason'] = qs['tokenized_overall_satisfaction_reason'].apply(lambda x: [item for item in x if item not in stop])
qs

,id,overall_satisfaction,overall_satisfaction_reason,tokenized_overall_satisfaction_reason
5,111377,2.0,店舗スタッフの対応が悪い,"[店舗, スタッフ, 対応, 悪い]"
7,111379,2.0,他のボドゲのお店だと店員や常連さんが気をきかせてくれていたので、そこと比べてしまうとどうして...,"[店員, 常連, きかせる, 比べる, どうしても, 多い]"
9,111381,2.0,想像していた人数よりも少ない。,"[想像, 人数, 少ない]"
11,111383,2.0,大人数なのにマンツーマン、フリータイムが少ない。,"[大, 人数, マンツーマン, フリー, タイム, 少ない]"
14,111386,2.0,食べ物が酒の肴にならない,"[食べ物, 酒, 肴]"
...,...,...,...,...
20295,131663,1.0,参加者が少なすぎる,"[参加, 少ない, すぎる]"
20296,131664,1.0,料理の品数が少なすぎた。\nスタッフの対応も最悪。,"[料理, 品数, 少ない, すぎる, スタッフ, 対応, 最悪]"
20307,131675,1.0,最終的に社長から返金対応する旨の提案を受け、案内を約束されたが未だ待っている状況。\n本当に...,"[最終, 社長, 返金, 対応, 旨, 提案, 受ける, 案内, 約束, 未だ, 待つ, 状..."
20308,131676,2.0,水上バスは夜景がきれいだったけど、ほんとにそれだけって感じだった。違和感がありすぎた。,"[水上, バス, 夜景, きれい, けど, ほんとに, だけ, 違和感, すぎる]"


## Generating Bigrams and Corpus

In [9]:
def bigrams(words, bi_min=15, tri_min=10):
    bigram = gensim.models.Phrases(words, min_count = bi_min)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return bigram_mod

In [10]:
def get_corpus(df):
    #df['text'] = strip_newline(df.text)
    words = stop
    bigram = bigrams(words)
    bigram = [bigram[review] for review in words]
    id2word = gensim.corpora.Dictionary(bigram)
    id2word.filter_extremes(no_below=10, no_above=0.35)
    id2word.compactify()
    corpus = [id2word.doc2bow(text) for text in bigram]
    return corpus, id2word, bigram

In [11]:
qs_new = qs[pd.notnull(qs['tokenized_overall_satisfaction_reason'])]
qs_new['tokenized_overall_satisfaction_reason'] = [' '.join(map(str, l)) for l in qs['tokenized_overall_satisfaction_reason']]
qs_new
print(qs_new['tokenized_overall_satisfaction_reason'])

5                                            店舗 スタッフ 対応 悪い
7                                  店員 常連 きかせる 比べる どうしても 多い
9                                                想像 人数 少ない
11                                 大 人数 マンツーマン フリー タイム 少ない
14                                                 食べ物 酒 肴
                               ...                        
20295                                           参加 少ない すぎる
20296                             料理 品数 少ない すぎる スタッフ 対応 最悪
20307    最終 社長 返金 対応 旨 提案 受ける 案内 約束 未だ 待つ 状況 本当に 返金 もらえ...
20308                      水上 バス 夜景 きれい けど ほんとに だけ 違和感 すぎる
20311                                   参加 費 交通 費 無駄 やる 詐欺
Name: tokenized_overall_satisfaction_reason, Length: 4410, dtype: object


In [12]:
data_satis = qs_new.tokenized_overall_satisfaction_reason.values.tolist()
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
data_words = list(sent_to_words(data_satis))
data_words[:5]

[['店舗', 'スタッフ', '対応', '悪い'],
 ['店員', '常連', 'きかせる', '比へる', 'とうしても', '多い'],
 ['想像', '人数', '少ない'],
 ['人数', 'マンツーマン', 'フリー', 'タイム', '少ない'],
 ['食へ物']]

In [13]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
print(trigram_mod[bigram_mod[data_words[0]]])

['店舗', 'スタッフ', '対応', '悪い']


In [14]:
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [15]:
data_words_bigrams = make_bigrams(data_words)
nlp = spacy.load('ja_ginza', disable=['parser', 'ner'])
nlp

In [16]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]
corpus

[[(0, 1), (1, 1), (2, 1), (3, 1)],
 [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)],
 [(10, 1), (11, 1), (12, 1)],
 [(10, 1), (11, 1), (13, 1), (14, 1)],
 [(15, 1)],
 [(16, 1), (17, 1), (18, 1), (19, 1), (20, 1)],
 [(6, 1),
  (10, 1),
  (21, 1),
  (22, 1),
  (23, 1),
  (24, 1),
  (25, 1),
  (26, 1),
  (27, 1),
  (28, 2),
  (29, 1),
  (30, 1),
  (31, 1),
  (32, 1),
  (33, 1),
  (34, 1),
  (35, 2),
  (36, 1),
  (37, 1),
  (38, 1),
  (39, 1),
  (40, 1),
  (41, 1),
  (42, 1),
  (43, 1),
  (44, 1),
  (45, 1),
  (46, 1),
  (47, 1),
  (48, 1),
  (49, 1),
  (50, 3),
  (51, 1),
  (52, 1),
  (53, 1),
  (54, 2),
  (55, 1)],
 [(34, 1),
  (56, 1),
  (57, 1),
  (58, 1),
  (59, 1),
  (60, 1),
  (61, 1),
  (62, 1),
  (63, 1),
  (64, 1),
  (65, 1),
  (66, 1),
  (67, 1),
  (68, 1),
  (69, 1),
  (70, 1),
  (71, 1),
  (72, 1)],
 [(26, 1), (73, 1), (74, 1), (75, 1)],
 [(10, 1), (11, 1), (45, 1), (54, 1), (76, 1), (77, 1), (78, 1)],
 [(3, 2),
  (10, 1),
  (11, 1),
  (33, 2),
  (41, 1),
  (42, 2),
  (45, 1)

In [17]:
size_id2word= [[(id2word[id], freq) for id, freq in cp] for cp in corpus]

In [18]:
print("Vocabulary size: {}".format(len(id2word)))
print("Corpus size: {}".format(len(corpus)))

Vocabulary size: 4389
Corpus size: 4410


## Create LDA Dissatisfaction_Reason TM

In [30]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, 
                                           id2word=id2word, 
                                           num_topics=6, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=100,
                                           alpha='auto',
                                           per_word_topics=True)
lda_model

In [31]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.040*"たけ" + 0.039*"行く" + 0.022*"二度と" + 0.020*"運営" + 0.019*"通り" + '
  '0.017*"テーフル" + 0.017*"案内" + 0.015*"くたさる" + 0.013*"正直" + 0.012*"知る"'),
 (1,
  '0.032*"くる" + 0.024*"出る" + 0.023*"食事" + 0.018*"友達" + 0.018*"進行" + 0.016*"説明" '
  '+ 0.014*"求める" + 0.014*"必要" + 0.014*"マッチンク" + 0.013*"言う"'),
 (2,
  '0.054*"スタッフ" + 0.034*"という" + 0.031*"対応" + 0.028*"出会い" + 0.024*"話す" + '
  '0.023*"悪い" + 0.020*"連絡" + 0.016*"交換" + 0.016*"たり" + 0.014*"話せる"'),
 (3,
  '0.076*"参加" + 0.061*"人数" + 0.053*"少ない" + 0.038*"すきる" + 0.036*"男性" + '
  '0.029*"女性" + 0.025*"イヘント" + 0.019*"料理" + 0.016*"男女" + 0.015*"コン"'),
 (4,
  '0.030*"最悪" + 0.028*"けと" + 0.028*"飲み物" + 0.028*"お金" + 0.025*"レヘル" + '
  '0.023*"無駄" + 0.022*"違う" + 0.019*"初めて" + 0.017*"食へる" + 0.016*"使う"')]


In [32]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus)) 
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -7.197125836502957

Coherence Score:  0.3979573426050968


In [22]:
#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
#vis

## Inferring The Categories From The Topics

In [23]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words):
    sent_topics_df = pd.DataFrame()
    
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row[0], key=lambda x: (x[1]), reverse=True)
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_contribution', 'Topic_Keywords']
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [24]:
categories_df = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data_words)
df_dominant_categories = categories_df.reset_index()
df_dominant_categories.columns = ['Document_No', 'Dominant_Categories', 'Category_perc_contrib', 'Keywords', 'Overall_Satisfaction_Reason']
df_dominant_categories

,Document_No,Dominant_Categories,Category_perc_contrib,Keywords,Overall_Satisfaction_Reason
0,0,2.0,0.2785,"スタッフ, 対応, 会場, 悪い, 残念, 違う, トリンク, 合う, 短い, 狭い","[店舗, スタッフ, 対応, 悪い]"
1,1,6.0,0.3343,"話す, 話せる, 会話, 全員, 記載, テーフル, 進行, 正直, 値段, 交流","[店員, 常連, きかせる, 比へる, とうしても, 多い]"
2,2,4.0,0.3444,"参加, 人数, 少ない, すきる, という, 料理, 出会い, たけ, 多い, 書く","[想像, 人数, 少ない]"
3,3,4.0,0.4086,"参加, 人数, 少ない, すきる, という, 料理, 出会い, たけ, 多い, 書く","[人数, マンツーマン, フリー, タイム, 少ない]"
4,4,4.0,0.3220,"参加, 人数, 少ない, すきる, という, 料理, 出会い, たけ, 多い, 書く",[食へ物]
...,...,...,...,...,...
4405,4405,4.0,0.4071,"参加, 人数, 少ない, すきる, という, 料理, 出会い, たけ, 多い, 書く","[参加, 少ない, すきる]"
4406,4406,4.0,0.3756,"参加, 人数, 少ない, すきる, という, 料理, 出会い, たけ, 多い, 書く","[料理, 品数, 少ない, すきる, スタッフ, 対応, 最悪]"
4407,4407,5.0,0.3167,"返金, 連絡, 交換, 最悪, 全く, 出来る, 本当に, 求める, 使う, 必要","[最終, 社長, 返金, 対応, 提案, 受ける, 案内, 約束, 未た, 待つ, 状況, ..."
4408,4408,4.0,0.3414,"参加, 人数, 少ない, すきる, という, 料理, 出会い, たけ, 多い, 書く","[水上, ハス, 夜景, きれい, けと, ほんとに, たけ, 違和感, すきる]"


In [25]:
#def qs_categories(model_type=lda_model, corpus=corpus, texts=data_words):
    #qs_cat_df = pd.DataFrame()
    #for i, row in enumerate(model_type[corpus]):
        #row = sorted(row[0], key= lambda x: (x[0]), reverse=False)
        #print(row)
    #topics_percentage_df = pd.DataFrame(row)
    #return topics_percentage_df
        #for j, (topic_num, prop_topic)in enumerate(row):
            #print(j, topic_num, prop_topic)
            #if j == 0:
                #wp = model_type.show_topic(topic_num)
                #topic_keywords = " ".join([word for word, prop in wp])
                #print(topic_keywords)
                #qs_cat_df = qs_cat_df.append(pd.Series([int(topic_num), round(prop_topic,2), round(prop_topic,2), round(prop_topic,2),round(prop_topic,2), round(prop_topic,2), topic_keywords]), ignore_index=True)
            #else:
                #break
    #qs_cat_df.columns = ['Dominant_Topic', 'Perc_Dom_Topic_0', 'Perc_Dom_Topic_1', 'Perc_Dom_Topic_2', 'Perc_Dom_Topic_3', 'Perc_Dom_Topic_4', 'Topic_Keywords']
    #contents = pd.Series(texts)
    #qs_cat_df = pd.concat([qs_cat_df, contents], axis=1)
    #return(qs_cat_df)

In [26]:
#qs_categories(model_type=lda_model, corpus=corpus, texts=data_words)
#categories_df = qs_categories(model_type=lda_model, corpus=corpus, texts=data_words)
#categories_df
#questionnaire_cat = categories_df.reset_index()
#questionnaire_cat.columns = ['Document_No', 'Dominant_Topic', 'Perc_Topic_0', 'Perc_Topic_1', 'Perc_Topic_2', 'Perc_Topic_3', 'Perc_Topic_4', 'Dom_Top_Keywords', 'Overall_Satisfaction_Reason']
#questionnaire_cat

In [27]:
#df_dominant_categories.Dominant_Categories.unique()
#df_dominant_categories.to_csv(r'/Users/jesse.ulundo/demo_tests/ov_sat_TM.csv', index = None, header=True)

In [34]:
category_code = {'参加人数に関して': 0, '女性と男性の出会い': 1, '参加者の年齢': 2, '会場と雰囲気': 3, 'スタッフ対応': 4, '飲食内容に関して': 5, 'その他': 6}
category_code

In [29]:
#qs_df['category_code'] = qs_df['category']
#qs_df = qs_df.replace({'category_code':category_codes})

NameError: name 'qs_df' is not defined

In [ ]:
#X_train, X_test, y_train, y_test = train_test_split(df['Overall_Satisfaction_Reason'], 
#                                                    df['Category_Code'], 
#                                                    test_size=0.15, 
#                                                    random_state=8)

In [ ]:
#Parameter election
#ngram_range = (1,2)
#min_df = 10
#max_df = 1.
#max_features = 300

In [ ]:
#tfidf = TfidfVectorizer(encoding='utf-8',
#                        ngram_range=ngram_range,
#                        stop_words=None,
#                        lowercase=False,
#                        max_df=max_df,
#                        min_df=min_df,
#                        max_features=max_features,
#                        norm='l2',
#                        sublinear_tf=True)
                        
#features_train = tfidf.fit_transform(X_train).toarray()
#labels_train = y_train
#print(features_train.shape)
#
#features_test = tfidf.transform(X_test).toarray()
#labels_test = y_test
#print(features_test.shape)

In [ ]:
#for Product, category_id in sorted(category_codes.items()):
#    features_chi2 = chi2(features_train, labels_train == category_id)
#    indices = np.argsort(features_chi2[0])
#    feature_names = np.array(tfidf.get_feature_names())[indices]
#    unigrams = [v for v in feature_names if len(v.split(' ')) == 1]
#    bigrams = [v for v in feature_names if len(v.split(' ')) == 2]
#    print("# '{}' category:".format(Product))
#    print("  . Most correlated unigrams:\n. {}".format('\n. '.join(unigrams[-5:])))
#    print("  . Most correlated bigrams:\n. {}".format('\n. '.join(bigrams[-2:])))
#    print("")